In [322]:
# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import scipy.stats as stats

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# get rid of warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%pylab inline
sns.set()

Populating the interactive namespace from numpy and matplotlib


In [3]:
# Load the data file (with new features already engineered)
df = pd.read_pickle('data/w_eng_features.pkl')

In [7]:
df.head(5)

,air_quality,rel_humidity,avg_num_clear_days,pct_area_water,rainfall_inches,temp_f,highest_point_ft,mean_elevation_ft,happiness_avg,HM__x__RF,TM__x__RF,HP__x__ME,pct_area_water_bc,mean_elevation_bc,hp_bc
Alabama,46.6,52.0,99.0,3.4,58.3,62.8,2413.0,500.0,53.72,3031.6,3661.24,1206500.0,1.045041,9.396474,19.679026
Alaska,29.1,64.0,61.0,14.2,22.5,26.6,20310.0,1900.0,54.60,1440.0,598.50,38589000.0,1.100227,12.555791,33.467698
Arizona,45.4,25.0,193.0,0.3,13.6,60.3,12637.0,4100.0,59.51,340.0,820.08,51811700.0,0.957583,14.630178,29.842066
Arkansas,43.1,49.0,123.0,2.1,50.6,60.4,2753.0,650.0,52.27,2479.4,3056.24,1789450.0,1.027070,9.976459,20.365059
California,46.0,62.0,146.0,4.8,22.2,59.4,14505.0,2900.0,62.81,1376.4,1318.68,42064500.0,1.058095,13.671538,30.858547


#### Setting up for modeling:

In [84]:
X = df.loc[:,['air_quality', 'rel_humidity', 'avg_num_clear_days',
                     'pct_area_water', 'rainfall_inches', 'temp_f',
                     'highest_point_ft', 'mean_elevation_ft', 'HM__x__RF',
                     'TM__x__RF', 'HP__x__ME', 'pct_area_water_bc', 
                      'mean_elevation_bc', 'hp_bc']]

y = df['happiness_avg']

# create overall quality squared term, which we expect to 
# help based on the relationship we see in the pair plot 

In [85]:
## Split the data 80 - 20 train_val/test

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.1)
# X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=.25, random_state=43)

In [86]:
## Scale the data
std = StandardScaler()
std.fit(X_train_val.values)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [87]:
## Scale the Predictors on both the train and test set
X_tr = std.transform(X_train_val.values)
X_te = std.transform(X_test.values)

## Run a LassoCV Model

In [200]:
# Run the cross validation, find the best alpha, refit the model on all the data with that alpha

alphavec_l = 10**np.linspace(-2,2,400)

lasso_model = LassoCV(alphas = alphavec_l, cv=5)
lasso_model.fit(X_tr, y_train_val)

LassoCV(alphas=array([1.00000e-02, 1.02335e-02, ..., 9.77181e+01, 1.00000e+02]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=None, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

In [201]:
# This is the best LASSO alpha value it found
lasso_model.alpha_

0.29763514416313175

In [202]:
list(zip(X_train_val.columns, lasso_model.coef_))

[('air_quality', -0.23903036947702255),
 ('rel_humidity', -0.0),
 ('avg_num_clear_days', 0.0),
 ('pct_area_water', 0.0),
 ('rainfall_inches', -0.0),
 ('temp_f', 0.0),
 ('highest_point_ft', -0.0),
 ('mean_elevation_ft', 0.0),
 ('HM__x__RF', -0.9266674406482377),
 ('TM__x__RF', -0.0),
 ('HP__x__ME', 0.0),
 ('pct_area_water_bc', 1.211661913410343),
 ('mean_elevation_bc', 0.7004106917018783),
 ('hp_bc', 0.0)]

In [203]:
# Make predictions on the test set using the LASSO model
test_set_pred_lasso = lasso_model.predict(X_te)

In [204]:
#Mean Absolute Error (MAE)
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true)) 

In [260]:
#Mean Squared Error (MSE)
def mse(y_true, y_pred):
    return np.mean(np.square(y_pred - y_true)) 

In [235]:
#Adjusted R-squared
def get_adj_r2(r2, n, p):
    return 1-(1-r2)*(n-1)/(n-1-p)

X.shape[0]

50

In [261]:
# Find the MSE and R^2 on the test set using this LASSO model
mse(y_test, test_set_pred_lasso)

2.792142244382581

In [250]:
# Find r2 score for LASSO
r2_lasso = r2_score(y_test, test_set_pred_lasso)
r2_lasso

0.5613591135282743

In [251]:
# Find adjusted r2 score for LASSO
adj_r2_lasso = get_adj_r2(r2_lasso, X.shape[0], X.shape[1])
adj_r2_lasso

0.38590275893958403

In [207]:
lasso_model

LassoCV(alphas=array([1.00000e-02, 1.02335e-02, ..., 9.77181e+01, 1.00000e+02]),
    copy_X=True, cv=5, eps=0.001, fit_intercept=True, max_iter=1000,
    n_alphas=100, n_jobs=None, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)

## Run a RidgeCV Model

In [314]:
# Run the cross validation, find the best alpha, refit the model on all the data with that alpha

# alphavec_r = 10**np.linspace(-2,0,400)
alphavec_r = np.logspace(-2,2,400)

ridge_model = RidgeCV(alphas = alphavec_r, cv=5)
ridge_model.fit(X_tr, y_train_val)

RidgeCV(alphas=array([1.00000e+00, 1.02335e+00, ..., 9.77181e+03, 1.00000e+04]),
    cv=5, fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)

In [315]:
# This is the best RIDGE alpha value found
ridge_model.alpha_

98.85245709128276

In [316]:
# These are the (standardized) coefficients found
# when it refit using that best alpha
list(zip(X_train_val.columns, ridge_model.coef_))

[('air_quality', -0.2144141117625792),
 ('rel_humidity', -0.0008763610258530456),
 ('avg_num_clear_days', 0.07024672410494597),
 ('pct_area_water', 0.21743838168867827),
 ('rainfall_inches', -0.18284990562504064),
 ('temp_f', -0.06746996051873774),
 ('highest_point_ft', 0.028193389025369706),
 ('mean_elevation_ft', 0.123068475814379),
 ('HM__x__RF', -0.17983927120667292),
 ('TM__x__RF', -0.15910062038959272),
 ('HP__x__ME', 0.06552191603010099),
 ('pct_area_water_bc', 0.28203619835508326),
 ('mean_elevation_bc', 0.17204039786179573),
 ('hp_bc', 0.10127310861189302)]

In [317]:
# Make predictions on the test set using the RIDGE model
test_set_pred_ridge = ridge_model.predict(X_te)

In [318]:
# Find the MSE and R^2 on the test set using this RIDGE model
# use mean squared error
mse(y_test, test_set_pred_ridge)

3.8733777205239512

In [319]:
# Find r2 score for RIDGE
r2_ridge = r2_score(y_test, test_set_pred_ridge)
r2_ridge

0.39149882482217235

In [320]:
# Find adjusted r2 score for LASSO
adj_r2_ridge = get_adj_r2(r2_ridge, X.shape[0], X.shape[1])
adj_r2_ridge

0.14809835475104138

# RidgeCV is being weird. Let's try GridSearch instead

In [324]:
# Grid Search for Algorithm Tuning

# Use the same list of alphas as I did for RidgeCV (alphavec_r)

# create and fit a ridge regression model, testing each alpha
model = Ridge()
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphavec_r))
grid.fit(X_tr, y_train_val)
print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_.alpha)

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'alpha': array([1.00000e+00, 1.02335e+00, ..., 9.77181e+03, 1.00000e+04])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)
-0.0040563871360431
88.07692733975462
